# 1. Installation

In [1]:
!pip install --upgrade pip
!pip install llmcompressor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 12.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 MB 15.3 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 15.5 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: nvidia-ml-py
    Found existing installation: nvidia-ml-py 13.590.48
    Uninstalling nvidia-ml-py-13.590.48:
      Successfully uninstalled nvidia-ml-py-13.590.480/9 [nvidia-ml-py]
  Attempting uninstall: dill0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/9 [pyarrow]
    Found existing installation: dill 0.4.1━━━━━━━━━━━━━━━━━━━ 2/9 [pyarrow]
    Uninstalling dill-0.4.1:0m╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/9 [dill]
      Successfully uninstalled dill-0.4.1━━━━━━━━━━━━━━━━━━━━━ 3/9 [dill]
  Attempting 

In [4]:
from huggingface_hub import login

!pip install ipywidgets

login()     # Your huggingface access token.

# 2. Compress Model

## 2.1. Quantization

See [Quantization Schemes](https://docs.vllm.ai/projects/llm-compressor/en/0.8.1/guides/compression_schemes/) for choosing quantization method.

EX1). `INT W8A8` quantization with SmoothQuant and GPTQ

In [ ]:
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot


recipe = [
    SmoothQuantModifier(smoothing_strength=0.8),
    GPTQModifier(scheme="W8A8", targets="Linear", ignore=["lm_head"]),
]


SAVE_DIR="TinyLlama-1.1B-Chat-v1.0-INT8"

oneshot(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset="open_platypus",
    recipe=recipe,
    output_dir=SAVE_DIR,
    max_seq_length=2048,
    num_calibration_samples=512,
)

In [ ]:
# Inference quantized model via vLLM
from vllm import LLM, SamplingParams


model_path = "./TinyLlama-1.1B-Chat-v1.0-INT8"
# model_path = SAVE_DIR

model = LLM(
    model=model_path,    
    gpu_memory_utilization=0.7, # GPU 메모리 70% 사용 (필요시 조절)
    tensor_parallel_size=1,   # GPU 1개 사용
    # enforce_eager=True      # 호환성 모드 켜기 (필요시)
)

sampling_params = SamplingParams(max_tokens=256)

outputs = model.generate("Sky is blue and Apple is ", sampling_params)

for output in outputs:
    print("Answer: ", output.outputs[0].text)

INFO 01-23 05:25:10 [utils.py:263] non-default args: {'gpu_memory_utilization': 0.7, 'disable_log_stats': True, 'model': './TinyLlama-1.1B-Chat-v1.0-INT8'}
INFO 01-23 05:25:10 [model.py:530] Resolved architecture: LlamaForCausalLM
INFO 01-23 05:25:10 [model.py:1545] Using max model len 2048
INFO 01-23 05:25:12 [scheduler.py:229] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-23 05:25:12 [vllm.py:630] Asynchronous scheduling is enabled.
INFO 01-23 05:25:12 [vllm.py:637] Disabling NCCL for DP synchronization when using async scheduling.
(EngineCore_DP0 pid=166254) INFO 01-23 05:25:12 [core.py:97] Initializing a V1 LLM engine (v0.14.0) with config: model='./TinyLlama-1.1B-Chat-v1.0-INT8', speculative_config=None, tokenizer='./TinyLlama-1.1B-Chat-v1.0-INT8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1,

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=166254) INFO 01-23 05:25:13 [default_loader.py:291] Loading weights took 0.20 seconds
(EngineCore_DP0 pid=166254) INFO 01-23 05:25:14 [gpu_model_runner.py:3905] Model loading took 1.15 GiB memory and 0.463936 seconds
(EngineCore_DP0 pid=166254) INFO 01-23 05:25:18 [backends.py:644] Using cache directory: /root/.cache/vllm/torch_compile_cache/c0332d605c/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=166254) INFO 01-23 05:25:18 [backends.py:704] Dynamo bytecode transform time: 4.53 s
(EngineCore_DP0 pid=166254) INFO 01-23 05:25:20 [backends.py:226] Directly load the compiled graph(s) for compile range (1, 8192) from the cache, took 0.332 s
(EngineCore_DP0 pid=166254) INFO 01-23 05:25:20 [monitor.py:34] torch.compile takes 4.86 s in total
(EngineCore_DP0 pid=166254) INFO 01-23 05:25:21 [gpu_worker.py:358] Available KV cache memory: 14.92 GiB
(EngineCore_DP0 pid=166254) INFO 01-23 05:25:21 [kv_cache_utils.py:1305] GPU KV cache size: 711,008 tokens
(Engin

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:00<00:00, 55.67it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:00<00:00, 54.45it/s]


(EngineCore_DP0 pid=166254) INFO 01-23 05:25:23 [gpu_model_runner.py:4856] Graph capturing finished in 2 secs, took 0.44 GiB
(EngineCore_DP0 pid=166254) INFO 01-23 05:25:23 [core.py:273] init engine (profile, create kv cache, warmup model) took 9.93 seconds
INFO 01-23 05:25:24 [llm.py:347] Supported tasks: ['generate']


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


2. And Pikachu is yellow and Google is red

So yes, those are some great questions and they present opportunities for social storytelling.


결과: 2.2Gb -> 1.15Gb로 원본대비 약 52% 압축

EX2). `INT W8A8` quantization with AWQ

In [1]:

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from llmcompressor import oneshot
from llmcompressor.modifiers.awq import AWQModifier
from llmcompressor.utils import dispatch_for_generation


def preprocess(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
        )
    }


# Tokenize inputs.
def tokenize(sample):
    return tokenizer(
        sample["text"],
        padding=False,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        add_special_tokens=False,
    )


# Select model and load it.
MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct"

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

# Select calibration dataset.
DATASET_ID = "HuggingFaceH4/ultrachat_200k"
DATASET_SPLIT = "train_sft"

# Select number of samples. 256 samples is a good place to start.
# Increasing the number of samples can improve accuracy.
NUM_CALIBRATION_SAMPLES = 256
MAX_SEQUENCE_LENGTH = 512

# Load dataset and preprocess.
ds = load_dataset(DATASET_ID, split=f"{DATASET_SPLIT}[:{NUM_CALIBRATION_SAMPLES}]")
ds = ds.shuffle(seed=47)
ds = ds.map(preprocess)

# Configure the quantization algorithm to run.
recipe = [
    AWQModifier(
        ignore=["lm_head"], scheme="W4A16_ASYM", targets=["Linear"], duo_scaling="both"
    ),
]

# Apply algorithms.
oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

# Save to disk compressed.
SAVE_DIR = MODEL_ID.rstrip("/").split("/")[-1] + "-awq-asym"
model.save_pretrained(SAVE_DIR, save_compressed=True)
tokenizer.save_pretrained(SAVE_DIR)

Tokenizing:   0%|          | 0/256 [00:00<?, ? examples/s]

2026-01-23T04:54:08.990687+0000 | reset | INFO - Compression lifecycle reset
2026-01-23T04:54:08.991631+0000 | from_modifiers | INFO - Creating recipe from modifiers
2026-01-23T04:54:09.005492+0000 | on_initialize | INFO - No AWQModifier.mappings provided, inferring from model...
2026-01-23T04:54:09.008350+0000 | _set_resolved_mappings | WARNING - skipping AWQ for model.layers.0.self_attn.v_proj for mapping AWQMapping(smooth_layer='re:.*v_proj$', balance_layers=['re:.*o_proj$']) because found incompatible balance layers
2026-01-23T04:54:09.008756+0000 | _set_resolved_mappings | WARNING - skipping AWQ for model.layers.1.self_attn.v_proj for mapping AWQMapping(smooth_layer='re:.*v_proj$', balance_layers=['re:.*o_proj$']) because found incompatible balance layers
2026-01-23T04:54:09.009110+0000 | _set_resolved_mappings | WARNING - skipping AWQ for model.layers.2.self_attn.v_proj for mapping AWQMapping(smooth_layer='re:.*v_proj$', balance_layers=['re:.*o_proj$']) because found incompatible

(17/17): Calibrating: 100%|██████████| 256/256 [00:00<00:00, 1154.81it/s]
Smoothing: 0it [00:00, ?it/s]
(17/17): Propagating: 100%|██████████| 256/256 [00:00<00:00, 2903.01it/s]
Smoothing: 0it [00:00, ?it/s]
Calibrating weights: 112it [00:00, 178.31it/s]

2026-01-23T04:57:16.070184+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers


2026-01-23T04:57:16.088555+0000 | post_process | WARNING - Optimized model is not saved. To save, please provide`output_dir` as input arg.Ex. `oneshot(..., output_dir=...)`
2026-01-23T04:57:16.103031+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 112it [00:01, 72.37it/s]


('Llama-3.2-1B-Instruct-awq-asym/tokenizer_config.json',
 'Llama-3.2-1B-Instruct-awq-asym/special_tokens_map.json',
 'Llama-3.2-1B-Instruct-awq-asym/chat_template.jinja',
 'Llama-3.2-1B-Instruct-awq-asym/tokenizer.json')

In [ ]:
from vllm import LLM, SamplingParams


model_path = SAVE_DIR

model = LLM(
    model=model_path,    
    gpu_memory_utilization=0.7, # GPU 메모리 70% 사용 (필요시 조절)
    tensor_parallel_size=1,   # GPU 1개 사용
    # enforce_eager=True      # 호환성 모드 켜기 (필요시)
)

prompt = "3 + 5 is "

sampling_params = SamplingParams(max_tokens=256)

outputs = model.generate(prompt, sampling_params)

for output in outputs:
    print(f"Prompt: {prompt}")
    print(f"Answer: ", output.outputs[0].text)

INFO 01-23 04:58:04 [utils.py:263] non-default args: {'gpu_memory_utilization': 0.7, 'disable_log_stats': True, 'model': 'Llama-3.2-1B-Instruct-awq-asym'}
INFO 01-23 04:58:04 [model.py:530] Resolved architecture: LlamaForCausalLM
INFO 01-23 04:58:04 [model.py:1545] Using max model len 131072
INFO 01-23 04:58:04 [scheduler.py:229] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-23 04:58:04 [vllm.py:630] Asynchronous scheduling is enabled.
INFO 01-23 04:58:04 [vllm.py:637] Disabling NCCL for DP synchronization when using async scheduling.


The tokenizer you are loading from 'Llama-3.2-1B-Instruct-awq-asym' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


WARNING 01-23 04:58:04 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
(EngineCore_DP0 pid=150803) INFO 01-23 04:58:09 [core.py:97] Initializing a V1 LLM engine (v0.14.0) with config: model='Llama-3.2-1B-Instruct-awq-asym', speculative_config=None, tokenizer='Llama-3.2-1B-Instruct-awq-asym', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=compressed-tensors, enforce_eager=False, enable_return_routed_experts=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  6.65it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  6.65it/s]
(EngineCore_DP0 pid=150803) 


(EngineCore_DP0 pid=150803) INFO 01-23 04:58:10 [default_loader.py:291] Loading weights took 0.18 seconds
(EngineCore_DP0 pid=150803) INFO 01-23 04:58:11 [gpu_model_runner.py:3905] Model loading took 0.99 GiB memory and 0.513573 seconds
(EngineCore_DP0 pid=150803) INFO 01-23 04:58:14 [backends.py:644] Using cache directory: /root/.cache/vllm/torch_compile_cache/38e4a3dd15/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=150803) INFO 01-23 04:58:14 [backends.py:704] Dynamo bytecode transform time: 3.01 s
(EngineCore_DP0 pid=150803) INFO 01-23 04:58:19 [backends.py:261] Cache the graph of compile range (1, 8192) for later use
(EngineCore_DP0 pid=150803) INFO 01-23 04:58:22 [backends.py:278] Compiling a graph for compile range (1, 8192) takes 5.32 s
(EngineCore_DP0 pid=150803) INFO 01-23 04:58:22 [monitor.py:34] torch.compile takes 8.33 s in total
(EngineCore_DP0 pid=150803) INFO 01-23 04:58:23 [gpu_worker.py:358] Available KV cache memory: 14.27 GiB
(EngineCore_DP0 pid=1508

...

(EngineCore_DP0 pid=150803) The tokenizer you are loading from 'Llama-3.2-1B-Instruct-awq-asym' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


(EngineCore_DP0 pid=150803) INFO 01-23 04:58:26 [vllm.py:630] Asynchronous scheduling is enabled.
INFO 01-23 04:58:26 [llm.py:347] Supported tasks: ['generate']


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 *ahem* I quote the opening line of the Angels & Demons novel...
"...legends and myths that had still begun to take shape in human minds as mode of keeping order. My own is rooted in science, which delivers the unpleasant truth, and, if used, it can provide incredible breakthroughs."
Are they human, or supernatural?
What connects them to the angels?
And what drives them?
What are the consequences of their combined efforts?
In this tale, do dark secrets differentiate a heroic sacrifice until it is insisted as the bànẹp items called The symbols that can blindly aver goKhistence and Passion remain basically identicalMen perce Di property presence wit.PMIN sau-ca = Enter guests activita
Cas trao cases workstationmarkedboth mại poate Consortium Ccccort Hosorian alternatively breasts sister v master tim iter thoroughly blindness Memade promote Zai des asked hom mixed lurking terms net export mature administration authority ord well mi durch Ack Plot carry business architecture_http Marbleเนc

결과: 2.47Gb -> 1.45Gb로 원본대비 약 42% 압축됨

-----

EX3). `INT W4A16` quantization with GPTQ 

In [2]:
# Load Model
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Load Calibration Dataset
from datasets import load_dataset

NUM_CALIBRATION_SAMPLES=512
MAX_SEQUENCE_LENGTH=2048

# Load dataset.
ds = load_dataset("HuggingFaceH4/ultrachat_200k", split=f"train_sft[:{NUM_CALIBRATION_SAMPLES}]")
ds = ds.shuffle(seed=42)

# Preprocess the data into the format the model is trained with.
def preprocess(example):
    return {"text": tokenizer.apply_chat_template(example["messages"], tokenize=False,)}
ds = ds.map(preprocess)

# Tokenize the data (be careful with bos tokens - we need add_special_tokens=False since the chat_template already added it).
def tokenize(sample):
    return tokenizer(sample["text"], padding=False, max_length=MAX_SEQUENCE_LENGTH, truncation=True, add_special_tokens=False)
ds = ds.map(tokenize, remove_columns=ds.column_names)

In [ ]:
# Apply quantization with GPTQ recipe
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier

# Configure the quantization algorithm to run.
recipe = GPTQModifier(targets="Linear", scheme="W4A16", ignore=["lm_head"])

# Save dir name
SAVE_DIR = MODEL_ID.rstrip("/").split("/")[-1] + "-W4A16-G128"

# Apply quantization.
oneshot(
    model=model, dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
    output_dir=SAVE_DIR,    # Tokenizer automatically saved together
)

# # Save to disk compressed.
# model.save_pretrained(SAVE_DIR, save_compressed=True)
# tokenizer.save_pretrained(SAVE_DIR)

2026-01-23T06:27:34.653254+0000 | reset | INFO - Compression lifecycle reset
2026-01-23T06:27:34.654706+0000 | from_modifiers | INFO - Creating recipe from modifiers
2026-01-23T06:27:34.680429+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2026-01-23T06:27:34.680893+0000 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `GPTQModifier`


(1/29): Calibrating: 100%|██████████| 512/512 [00:10<00:00, 48.27it/s]

2026-01-23T06:27:45.966714+0000 | compress_modules | INFO - Quantizing model.layers.0.self_attn.q_proj using 512 samples


2026-01-23T06:27:47.040387+0000 | compress | METRIC - time 1.07s
2026-01-23T06:27:47.040829+0000 | compress | METRIC - error 918.60
2026-01-23T06:27:47.041510+0000 | compress | METRIC - GPU 0 | usage: 12.24% | total memory: 25 GB
2026-01-23T06:27:47.041805+0000 | compress | METRIC - Compressed module size: 19.095552 MB
2026-01-23T06:27:47.042267+0000 | compress_modules | INFO - Quantizing model.layers.0.self_attn.k_proj using 512 samples
2026-01-23T06:27:47.921058+0000 | compress | METRIC - time 0.88s
2026-01-23T06:27:47.921498+0000 | compress | METRIC - error 466.25
2026-01-23T06:27:47.921929+0000 | compress | METRIC - GPU 0 | usage: 12.24% | total memory: 25 GB
2026-01-23T06:27:47.922168+0000 | compress | METRIC - Compressed module size: 6.365184 MB
2026-01-23T06:27:47.922571+0000 | compress_modules | INFO - Quantizing model.layers.0.self_attn.v_proj using 512 samples
2026-01-23T06:27:48.776163+0000 | compress | METRIC - time 0.85s
2026-01-23T06:27:48.776801+0000 | compress | METRIC 

(2/29): Calibrating: 100%|██████████| 512/512 [00:10<00:00, 48.34it/s]

...

(29/29): Propagating: 100%|██████████| 512/512 [00:00<00:00, 1444.29it/s]

2026-01-23T06:37:56.158784+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers


2026-01-23T06:37:56.188614+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 196it [00:05, 36.26it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (

결과: 6.43Gb -> 2.82Gb로 약 44% 압축

In [ ]:
# If you need.
!pip install -U lm_eval       

In [5]:
# our Quantized model eval
!lm_eval --model vllm \
  --model_args pretrained="./Llama-3.2-3B-Instruct-W4A16-G128",add_bos_token=true \
  --tasks gsm8k \
  --num_fewshot 5 \
  --limit 250 \
  --batch_size 'auto'

[2026-01-23 06:38:44] WARNING __main__.py:369:  --limit SHOULD ONLY BE USED FOR TESTING.REAL METRICS SHOULD NOT BE COMPUTED USING LIMIT.
[2026-01-23 06:38:44] INFO __main__.py:465: Selected Tasks: ['gsm8k']
[2026-01-23 06:38:44] WARNING evaluator.py:172: pretrained=pretrained=./Llama-3.2-3B-Instruct-W4A16-G128,add_bos_token=true appears to be an instruct or chat variant but chat template is
        not applied. Recommend setting `apply_chat_template` (optionally `fewshot_as_multiturn`).
[2026-01-23 06:38:44] INFO evaluator.py:202: Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
[2026-01-23 06:38:44] INFO evaluator.py:240: Initializing vllm model, with arguments: {'pretrained': './Llama-3.2-3B-Instruct-W4A16-G128', 'add_bos_token': True}
INFO 01-23 06:38:44 [utils.py:263] non-default args: {'seed': 1234, 'disable_log_stats': True, 'model': './Llama-3.2-3B-Instruct-W4A16-G128'}
INFO 01-23 06:38:44 [model.py:5

In [ ]:
# Original model eval
!lm_eval --model vllm \
  --model_args pretrained="meta-llama/Llama-3.2-3B-Instruct",add_bos_token=true,dtype=auto,max_model_len=4096 \
  --tasks gsm8k \
  --num_fewshot 5 \
  --limit 250 \
  --batch_size 'auto'

**Quantized Model**

|Tasks|Version|     Filter     |n-shot|  Metric   |   |Value|   |Stderr|
|-----|------:|----------------|-----:|-----------|---|----:|---|-----:|
|gsm8k|      3|flexible-extract|     5|exact_match|↑  | 0.64|±  |0.0304|
|     |       |strict-match    |     5|exact_match|↑  | 0.54|±  |0.0316|

**Original Model**

|Tasks|Version|     Filter     |n-shot|  Metric   |   |Value|   |Stderr|
|-----|------:|----------------|-----:|-----------|---|----:|---|-----:|
|gsm8k|      3|flexible-extract|     5|exact_match|↑  |0.680|±  |0.0296|
|     |       |strict-match    |     5|exact_match|↑  |0.608|±  |0.0309|

-----


## 2.2. Pruning

2:4 sparse pruning with/without FP8 quantization

In [1]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from llmcompressor import oneshot
from llmcompressor.modifiers.obcq import SparseGPTModifier
from llmcompressor.modifiers.quantization import QuantizationModifier
from llmcompressor.utils import dispatch_for_generation


# Configuration
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
DATASET_ID = "HuggingFaceH4/ultrachat_200k"
DATASET_SPLIT = "train_sft"
NUM_CALIBRATION_SAMPLES = 512
MAX_SEQUENCE_LENGTH = 2048
QUANT_ENABLE = False

def preprocess(example):
    """Preprocess dataset examples."""
    return {"text": tokenizer.apply_chat_template(example["messages"], tokenize=False)}


def tokenize(sample):
    """Tokenize dataset examples."""
    return tokenizer(
        sample["text"],
        padding=False,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        add_special_tokens=False,
    )


def get_recipe(fp8_enabled):
    """
    Generate the compression recipe and save directory based on the FP8 flag.
    """
    
    base_recipe = [
        SparseGPTModifier(
            sparsity=0.5,
            mask_structure="2:4",
            targets=[r"re:model.layers.\d*$"],
        )
    ]
    save_dir = MODEL_ID.rstrip("/").split("/")[-1] + "2of4-sparse"

    if fp8_enabled:
        base_recipe.append(
            QuantizationModifier(
                targets=["Linear"],
                ignore=["lm_head"],
                scheme="FP8_DYNAMIC",
            )
        )
        save_dir = (
            MODEL_ID.rstrip("/").split("/")[-1] + "2of4-W8A8-FP8-Dynamic-Per-Token"
        )

        # check that asymmetric quantization is not being used
        q_scheme = base_recipe[1].scheme
        if not isinstance(q_scheme, str) and not q_scheme["weights"].symmetric:
            raise ValueError(
                "Asymmetric quantization with 2of4 sparsity is not supported by vLLM. "
                "Please use symmetric quantization"
            )

    return base_recipe, save_dir

Prune model

In [2]:
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, dtype='auto')
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# Load and preprocess dataset
ds = load_dataset(
    DATASET_ID, 
    split=f"{DATASET_SPLIT}[:{NUM_CALIBRATION_SAMPLES}]"
).shuffle(seed=47)
ds = ds.map(preprocess)
ds = ds.map(tokenize, remove_columns=ds.column_names)

# Get compression recipe and save directory
recipe, save_dir = get_recipe(QUANT_ENABLE)

# Apply compression
oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
    output_dir = save_dir
)

# Validate the compressed model
print("\n========== SAMPLE GENERATION ==============")
dispatch_for_generation(model)
input_ids = tokenizer("Hello my name is", return_tensors="pt").input_ids.to(
    model.device
)
output = model.generate(input_ids, max_new_tokens=100)
print(tokenizer.decode(output[0]))
print("==========================================\n")

# # Save compressed model and tokenizer
# model.save_pretrained(save_dir)
# tokenizer.save_pretrained(save_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_1398/4116749871.py:40: DeprecationWarning: SparseGPTModifier has moved. In future, please initialize it from `llmcompressor.modifiers.pruning.sparsegpt.SparseGPTModifier`.
  SparseGPTModifier(


2026-01-27T04:00:17.572250+0000 | reset | INFO - Compression lifecycle reset
2026-01-27T04:00:17.573653+0000 | from_modifiers | INFO - Creating recipe from modifiers
2026-01-27T04:00:17.579295+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2026-01-27T04:00:17.579625+0000 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `SparseGPTModifier`
2026-01-27T04:00:17.723764+0000 | get_sequential_targets | WARNING - Passing sequential targets through modifiers is deprecated, please use `oneshot(sequential_targets=...)`


(1/29): Calibrating: 100%|██████████| 512/512 [00:10<00:00, 47.96it/s]

2026-01-27T04:00:28.903087+0000 | compress_modules | INFO - Sparsifying model.layers.0.self_attn.q_proj using 512 samples


...

2026-01-27T04:08:58.902434+0000 | is_sparse24_bitmask_supported | WARNING - Compressed Sparse-only 2:4 models are not supported in vLLM<=0.7.0, consider saving with `disable_sparse_compression` set, `model.save_pretrained(..., disable_sparse_compression=True)`



Compressing model: 196it [00:06, 29.52it/s]
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.



========== SAMPLE GENERATION ==============


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


AttributeError: 'Linear' object has no attribute 'weight'

In [ ]:
# Inference pruned model via vLLM
from vllm import LLM, SamplingParams


model_path = r"Llama-3.2-3B-Instruct2of4-sparse"

model = LLM(
    model=model_path,    
    gpu_memory_utilization=0.7, # GPU 메모리 70% 사용 (필요시 조절)
    tensor_parallel_size=1,   # GPU 1개 사용
    enforce_eager=True,      # 호환성 모드 켜기 (필요시)
    dtype='auto'
)

sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=256
    )

prompt = "Hello, My name is:"

outputs = model.generate(prompt, sampling_params)

for output in outputs:
    print(output.outputs[0].text)